# **面试题目**

>## 解答以下题目，每道题有对应的分值

>## <span style="color:red;">注意：50分为通过，时间限制为40min,可查阅资料**</span>

<span style="color:blue;">**请执行以下命令下载样本数据**</span>

数据为CSV表格有8列，其中abstime列为时间列，其余列为特征列

In [1]:
!wget http://82.156.209.173/s/dHCnaH32rtZdxcs/download
!mv download sample_data_A100159.csv

--2022-04-19 08:27:17--  http://82.156.209.173/s/dHCnaH32rtZdxcs/download
正在连接 82.156.209.173:80... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度：211463 (207K) [text/csv]
正在保存至: “download”

download            100%[===================>] 206.51K  --.-KB/s  用时 0.1s      

2022-04-19 08:27:17 (2.10 MB/s) - 已保存 “download” [211463/211463])



### <span style="color:green;">1. **以下代码为基于MLP网络的训练模型。**</span>
### **该题考查pytorch基础及简单网络搭建，分值为30分**
  1. 要求实现get_training_model函数，在函数中构建3层MLP网络（4-8-8-3），输入层是4，连接层为8，连接层为8，分类数为3 ，以函数get_training_model()返回模型。
  2. 补全第69、70、71行代码，使准确率提升到99%。
  3. 代码详见：code_of_Q4.py
>提示：用pytorch构建3层MLP网络（网络使用relu激活，其余用线性层连接）。

In [12]:

class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
#         self.linear1 = torch.nn.Linear(8, 4)
        self.linear1 = torch.nn.Linear(4, 8)
        self.linear2 = torch.nn.Linear(8, 8)  # 搭建一下每一层的线性结构
        self.linear3 = torch.nn.Linear(8, 3)
        self.s = torch.nn.ReLU()  # 这里的Sigmoid是一个模块

        
    def forward(self, x):
        x = self.s(self.linear1(x))  # 用每一层的线性结构进行非线性输出
        x = self.s(self.linear2(x))  # !!!!这里不能用 self.sigmoid命名 会发生冲突报错
        x = self.s(self.linear3(x))
        return x



def get_training_model():
    #################################
    #  CODING HERE
    #################################
    model = Model()
    return model

In [16]:
# code_of_Q4.py
#import mlp
from torch.optim import SGD
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
import torch.nn as nn
import torch


def next_batch(inputs, targets, batchSize):
    # loop over the dataset
    for i in range(0, inputs.shape[0], batchSize):
        # yield a tuple of the current batched data and labels
        yield (inputs[i:i + batchSize], targets[i:i + batchSize])

# specify our batch size, number of epochs, and learning rate
BATCH_SIZE = 64
EPOCHS = 1000
LR = 1e-2

# determine the device we will be using for training
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("[INFO] training using {}...".format(DEVICE))

# generate a 3-class classification problem with 1000 data points,
# where each data point is a 4D feature vector
print("[INFO] preparing data...")
(X, y) = make_blobs(n_samples=1000, n_features=4, centers=3,
                    cluster_std=2.5, random_state=95)

# create training and testing splits, and convert them to PyTorch
# tensors
(trainX, testX, trainY, testY) = train_test_split(X, y,
                                                  test_size=0.15, random_state=95)
trainX = torch.from_numpy(trainX).float()
testX = torch.from_numpy(testX).float()
trainY = torch.from_numpy(trainY).float()
testY = torch.from_numpy(testY).float()

# initialize our model and display its architecture
mlp = get_training_model().to(DEVICE)

# initialize optimizer and loss function
opt = SGD(mlp.parameters(), lr=LR)
lossFunc = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(),lr=0.001)
# create a template to summarize current training progress
trainTemplate = "epoch: {} test loss: {:.3f} test accuracy: {:.3f}"

# loop through the epochs
for epoch in range(0, EPOCHS):
    # initialize tracker variables and set our model to trainable
    print("[INFO] epoch: {}...".format(epoch + 1))
    trainLoss = 0
    trainAcc = 0
    samples = 0
    mlp.train()

    # loop over the current batch of data
    for (batchX, batchY) in next_batch(trainX, trainY, BATCH_SIZE):
        # flash data to the current device, run it through our
        # model, and calculate loss
        (batchX, batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))
        predictions = mlp(batchX)
        loss = lossFunc(predictions, batchY.long())
        
        

        # zero the gradients accumulated from the previous steps,
        opt.zero_grad()  # 将梯度置零 如果不删除相当于batch的累加，每次求的梯度就会累加，比如上一次计算的梯度
        # perform backpropagation, and update model parameters
#         loss.backward()  # 计算当前loss对权重的梯度
        #################################
        #  CODING HERE
        #################################
      
        print(epoch, loss.item())  # 去掉格式，直接输出数据

        # Backward
        
        # 为a，那么下一次的梯度就是a+b（b是下一次的梯度，但是最终的结果会累加）
        loss.backward()  # 计算当前loss对权重的梯度
        
        opt.step()

        
        
        
        # update training loss, accuracy, and the number of samples
        # visited
        trainLoss += loss.item() * batchY.size(0)
        trainAcc += (predictions.max(1)[1] == batchY).sum().item()
        samples += batchY.size(0)

    # display model progress on the current training batch
    trainTemplate = "epoch: {} train loss: {:.3f} train accuracy: {:.3f}"
    print(trainTemplate.format(epoch + 1, (trainLoss / samples),
                               (trainAcc / samples)))

    # initialize tracker variables for testing, then set our model to
    # evaluation mode
    testLoss = 0
    testAcc = 0
    samples = 0
    mlp.eval()

    # initialize a no-gradient context
    with torch.no_grad():
        # loop over the current batch of test data
        for (batchX, batchY) in next_batch(testX, testY, BATCH_SIZE):
            # flash the data to the current device
            (batchX, batchY) = (batchX.to(DEVICE), batchY.to(DEVICE))

            # run data through our model and calculate loss
            predictions = mlp(batchX)
            loss = lossFunc(predictions, batchY.long())

            # update test loss, accuracy, and the number of
            # samples visited
            testLoss += loss.item() * batchY.size(0)
            testAcc += (predictions.max(1)[1] == batchY).sum().item()
            samples += batchY.size(0)

        # display model progress on the current test batch
        testTemplate = "epoch: {} test loss: {:.3f} test accuracy: {:.3f}"
        print(testTemplate.format(epoch + 1, (testLoss / samples),
                                  (testAcc / samples)))
        print("")



[INFO] training using cpu...
[INFO] preparing data...
[INFO] epoch: 1...
0 0.9751648902893066
0 1.094709038734436
0 1.0377061367034912
0 1.0679523944854736
0 1.0435118675231934
0 0.9330349564552307
0 1.0109739303588867
0 1.0402185916900635
0 1.0195488929748535
0 1.0328062772750854
0 0.8899589776992798
0 0.9176490306854248
0 0.9328872561454773
0 0.9614986777305603
epoch: 1 train loss: 0.999 train accuracy: 0.334
epoch: 1 test loss: 0.953 test accuracy: 0.347

[INFO] epoch: 2...
1 0.8648512959480286
1 0.9810410737991333
1 0.9352983236312866
1 0.9739397764205933
1 0.955819308757782
1 0.8489758372306824
1 0.9261025190353394
1 0.9588552713394165
1 0.9345546364784241
1 0.9531240463256836
1 0.8183751702308655
1 0.8451938629150391
1 0.8632398247718811
1 0.9029595255851746
epoch: 2 train loss: 0.912 train accuracy: 0.365
epoch: 2 test loss: 0.891 test accuracy: 0.393

[INFO] epoch: 3...
2 0.7956065535545349
2 0.9145874977111816
2 0.8723772168159485
2 0.9145123958587646
2 0.901840090751648
2 0.7

33 0.18532297015190125
33 0.14291195571422577
epoch: 34 train loss: 0.191 train accuracy: 0.959
epoch: 34 test loss: 0.169 test accuracy: 0.973

[INFO] epoch: 35...
34 0.1185159981250763
34 0.15268386900424957
34 0.21755896508693695
34 0.209917813539505
34 0.2064274400472641
34 0.18975183367729187
34 0.19614186882972717
34 0.20265352725982666
34 0.16937300562858582
34 0.20815999805927277
34 0.16025155782699585
34 0.11041031777858734
34 0.17212098836898804
34 0.12611395120620728
epoch: 35 train loss: 0.177 train accuracy: 0.958
epoch: 35 test loss: 0.154 test accuracy: 0.980

[INFO] epoch: 36...
35 0.10759228467941284
35 0.14097820222377777
35 0.20277881622314453
35 0.19667144119739532
35 0.19102956354618073
35 0.17915135622024536
35 0.18482738733291626
35 0.1896744817495346
35 0.15620413422584534
35 0.19207701086997986
35 0.14703768491744995
35 0.09884712845087051
35 0.1603844165802002
35 0.11114471405744553
epoch: 36 train loss: 0.164 train accuracy: 0.959
epoch: 36 test loss: 0.141 t

67 0.010798422619700432
epoch: 68 train loss: 0.055 train accuracy: 0.979
epoch: 68 test loss: 0.053 test accuracy: 0.993

[INFO] epoch: 69...
68 0.014165371656417847
68 0.05007161200046539
68 0.08405525237321854
68 0.07099370658397675
68 0.05504052713513374
68 0.06979458034038544
68 0.1120915338397026
68 0.07433062791824341
68 0.03580194339156151
68 0.04776737838983536
68 0.04504474252462387
68 0.011087536811828613
68 0.05320262908935547
68 0.01039531733840704
epoch: 69 train loss: 0.055 train accuracy: 0.979
epoch: 69 test loss: 0.052 test accuracy: 0.993

[INFO] epoch: 70...
69 0.013718876987695694
69 0.049608342349529266
69 0.08340496569871902
69 0.07009207457304001
69 0.0542265921831131
69 0.06905050575733185
69 0.11215028911828995
69 0.0735815241932869
69 0.035203967243433
69 0.04686654359102249
69 0.044599443674087524
69 0.010741401463747025
69 0.052371568977832794
69 0.010016991756856441
epoch: 70 train loss: 0.054 train accuracy: 0.979
epoch: 70 test loss: 0.052 test accuracy:

102 0.053385816514492035
102 0.03978392109274864
102 0.05554942786693573
102 0.11733422428369522
102 0.05947771668434143
102 0.02634160779416561
102 0.031166281551122665
102 0.03648224472999573
102 0.005748879164457321
102 0.038619935512542725
102 0.00430078711360693
epoch: 103 train loss: 0.044 train accuracy: 0.984
epoch: 103 test loss: 0.047 test accuracy: 0.993

[INFO] epoch: 104...
103 0.00687421066686511
103 0.039813753217458725
103 0.07095348834991455
103 0.053078871220350266
103 0.03950851783156395
103 0.055314160883426666
103 0.11748097091913223
103 0.0592174232006073
103 0.02622256986796856
103 0.030912315472960472
103 0.03632238134741783
103 0.005684775300323963
103 0.0383804552257061
103 0.004225105047225952
epoch: 104 train loss: 0.044 train accuracy: 0.984
epoch: 104 test loss: 0.047 test accuracy: 0.993

[INFO] epoch: 105...
104 0.006784422788769007
104 0.039616599678993225
104 0.07071524113416672
104 0.052779048681259155
104 0.039238348603248596
104 0.0550849549472332
1

136 0.05270254611968994
136 0.023712657392024994
136 0.02477685734629631
136 0.032027799636125565
136 0.00438560638576746
136 0.0325884111225605
136 0.002684329403564334
epoch: 137 train loss: 0.039 train accuracy: 0.984
epoch: 137 test loss: 0.045 test accuracy: 0.993

[INFO] epoch: 138...
137 0.004923957400023937
137 0.03429042547941208
137 0.06401325762271881
137 0.045313701033592224
137 0.031820785254240036
137 0.04974688962101936
137 0.12134570628404617
137 0.05255464091897011
137 0.023662198334932327
137 0.024631548672914505
137 0.03191437944769859
137 0.004361005499958992
137 0.03245316445827484
137 0.002655316609889269
epoch: 138 train loss: 0.039 train accuracy: 0.984
epoch: 138 test loss: 0.045 test accuracy: 0.993

[INFO] epoch: 139...
138 0.004887899849563837
138 0.034153591841459274
138 0.06383360922336578
138 0.04513851925730705
138 0.03163193166255951
138 0.0496356301009655
138 0.12142492085695267
138 0.05240793153643608
138 0.023612825199961662
138 0.02448865957558155
1

...

843 0.004014029633253813
843 0.012390196323394775
843 0.01848529279232025
843 0.01821986213326454
843 0.006787595804780722
843 0.017946511507034302
843 0.11241868138313293
843 0.00889559742063284
843 0.012646286748349667
843 0.005538288503885269
843 0.006060453597456217
843 0.0006970185786485672
843 0.00793678592890501
843 0.0006386562599800527
epoch: 844 train loss: 0.017 train accuracy: 0.996
epoch: 844 test loss: 0.032 test accuracy: 0.993

[INFO] epoch: 845...
844 0.004021044820547104
844 0.012438029050827026
844 0.018457874655723572
844 0.018217263743281364
844 0.0067933183163404465
844 0.017926106229424477
844 0.11226077377796173
844 0.008894520811736584
844 0.012605492025613785
844 0.005534596741199493
844 0.0060440246015787125
844 0.0006949300295673311
844 0.00791124813258648
844 0.0006412898655980825
epoch: 845 train loss: 0.017 train accuracy: 0.996
epoch: 845 test loss: 0.032 test accuracy: 0.993

[INFO] epoch: 846...
845 0.00402773916721344
845 0.012379070743918419
845 0.01

877 0.0006228158599697053
epoch: 878 train loss: 0.017 train accuracy: 0.996
epoch: 878 test loss: 0.032 test accuracy: 0.993

[INFO] epoch: 879...
878 0.004136369097977877
878 0.012182885780930519
878 0.01797763630747795
878 0.01779252663254738
878 0.006709705572575331
878 0.017417212948203087
878 0.11187676340341568
878 0.008406175300478935
878 0.012305370531976223
878 0.005338851362466812
878 0.005694367457181215
878 0.0006689273868687451
878 0.00759857939556241
878 0.000625804765149951
epoch: 879 train loss: 0.017 train accuracy: 0.996
epoch: 879 test loss: 0.032 test accuracy: 0.993

[INFO] epoch: 880...
879 0.0041450997814536095
879 0.012121857143938541
879 0.017967501655220985
879 0.017790822312235832
879 0.0067023783922195435
879 0.017397116869688034
879 0.1119745671749115
879 0.00837424211204052
879 0.012354392558336258
879 0.005333997774869204
879 0.005686931777745485
879 0.0006691639428026974
879 0.007615272421389818
879 0.0006212500156834722
epoch: 880 train loss: 0.017 tra

912 0.006651639472693205
912 0.016931628808379173
912 0.11127637326717377
912 0.007952284999191761
912 0.012071872130036354
912 0.005155561026185751
912 0.005368636921048164
912 0.0006466329796239734
912 0.007331185508519411
912 0.000605734356213361
epoch: 913 train loss: 0.017 train accuracy: 0.996
epoch: 913 test loss: 0.032 test accuracy: 0.993

[INFO] epoch: 914...
913 0.004268238320946693
913 0.011918708682060242
913 0.017483003437519073
913 0.017395786941051483
913 0.006643453612923622
913 0.01691312901675701
913 0.11132124811410904
913 0.00792053621262312
913 0.012120785191655159
913 0.005150717217475176
913 0.005361857824027538
913 0.0006470635416917503
913 0.007347777485847473
913 0.0006011656951159239
epoch: 914 train loss: 0.017 train accuracy: 0.996
epoch: 914 test loss: 0.032 test accuracy: 0.993

[INFO] epoch: 915...
914 0.004274576902389526
914 0.011974841356277466
914 0.01745702512562275
914 0.017392650246620178
914 0.00665309326723218
914 0.016895372420549393
914 0.111

946 0.0005860677920281887
epoch: 947 train loss: 0.017 train accuracy: 0.996
epoch: 947 test loss: 0.032 test accuracy: 0.993

[INFO] epoch: 948...
947 0.004376339726150036
947 0.01173239853233099
947 0.01700727455317974
947 0.017025122418999672
947 0.006596815772354603
947 0.01646764576435089
947 0.11062904447317123
947 0.007485934998840094
947 0.011881224811077118
947 0.004974684212356806
947 0.005066439509391785
947 0.0006275693885982037
947 0.007101227063685656
947 0.0005789455026388168
epoch: 948 train loss: 0.017 train accuracy: 0.996
epoch: 948 test loss: 0.032 test accuracy: 0.993

[INFO] epoch: 949...
948 0.004376269411295652
948 0.0118323415517807
948 0.01698119565844536
948 0.017001880332827568
948 0.006603112444281578
948 0.016450954601168633
948 0.11058555543422699
948 0.007494404446333647
948 0.011830267496407032
948 0.0049674431793391705
948 0.005047711543738842
948 0.0006264067487791181
948 0.00707081938162446
948 0.0005826801061630249
epoch: 949 train loss: 0.017 train

980 0.011563252657651901
980 0.004819859750568867
980 0.0047777206636965275
980 0.0006075970595702529
980 0.0068248868919909
980 0.0005684063653461635
epoch: 981 train loss: 0.016 train accuracy: 0.996
epoch: 981 test loss: 0.032 test accuracy: 0.993

[INFO] epoch: 982...
981 0.004491779487580061
981 0.011535098776221275
981 0.016582494601607323
981 0.016701070591807365
981 0.006541272159665823
981 0.016029950231313705
981 0.10999462753534317
981 0.0071044014766812325
981 0.011628844775259495
981 0.0048192632384598255
981 0.004777571652084589
981 0.00060772750293836
981 0.006852135527879
981 0.0005621471209451556
epoch: 982 train loss: 0.016 train accuracy: 0.996
epoch: 982 test loss: 0.032 test accuracy: 0.993

[INFO] epoch: 983...
982 0.004498870112001896
982 0.01161322370171547
982 0.016567736864089966
982 0.016675742343068123
982 0.00655833724886179
982 0.016015110537409782
982 0.10990024358034134
982 0.007112548686563969
982 0.01158164907246828
982 0.004812790546566248
982 0.00477

### <span style="color:green;">2. **对给出的样本数据实现时间维度上的滑窗，设置窗长为64 窗移为30，窗宽为样本数据的维度， 将文件切分成n个大小一样的子文件并以csv格式保存到指定的文件夹，将实现的代码封装成函数。</span>**
### **该题考查数据预处理基础，函数实现及调用，分值为20分**
>提示滑窗类似于卷积核的移动，窗长为卷积核的长度，窗移为移动的步长

In [21]:
import pandas as pd 
import numpy as np 
import random 
import os
from PIL import Image 
sample_data = pd.read_csv("sample_data_A100159.csv")
print(sample_data)

def sliding_window(input_file='sample_data_A100159.csv', output_path='output', window_len=64, window_move=30): 
    """ 
    Args:
        input_path:输⼊⽂件路径, 
        output_path:输出⽂件路径, 
        window_len:窗⻓, 
        window_move:窗移 """
    if not os.path.exists(output_path):
        os.mkdir( output_path)
    #################################
#     if len（sample_data
    
    sample_data = pd.read_csv("sample_data_A100159.csv")
    for i in range(0,len(sample_data),window_move):
        df_sample = sample_data.iloc[i:i+window_len,:] 
        if len(df_sample) == 64:
            """最后一个文件如果小于64要处理么"""
            df_sample.to_csv("{}/input_file_{}.csv".format(output_path,i),index=0)
        
        """最后一个文件如果小于64要处理么"""
    #  CODING HERE
    #################################
    pass

sliding_window()

      BMS_BattCurr  BMS_RMC_CellVoltMax  BMS_SOC  BMS_BattVolt  \
0              0.0                 3.64     36.0         334.5   
1              0.0                 3.64     36.0         334.5   
2              0.0                 3.64     36.0         334.5   
3              0.0                 3.64     36.0         334.5   
4              0.0                 3.64     36.0         334.5   
...            ...                  ...      ...           ...   
4900         -26.0                 4.23     99.0         388.2   
4901         -26.0                 4.23     99.0         388.5   
4902         -25.9                 4.23     99.0         388.2   
4903         -25.8                 4.23    100.0         388.2   
4904         -14.0                 4.23    100.0         388.0   

      BMS_RMC_ModuleTempMax  BMS_RMC_ModuleTempMin  abstime  VehTotDistance  
0                      31.0                   28.0        0           10000  
1                      31.0                   28.0 

In [22]:
!ls output/

input_file_0.csv    input_file_2100.csv input_file_3210.csv input_file_4320.csv
input_file_1020.csv input_file_2130.csv input_file_3240.csv input_file_4350.csv
input_file_1050.csv input_file_2160.csv input_file_3270.csv input_file_4380.csv
input_file_1080.csv input_file_2190.csv input_file_330.csv  input_file_4410.csv
input_file_1110.csv input_file_2220.csv input_file_3300.csv input_file_4440.csv
input_file_1140.csv input_file_2250.csv input_file_3330.csv input_file_4470.csv
input_file_1170.csv input_file_2280.csv input_file_3360.csv input_file_450.csv
input_file_120.csv  input_file_2310.csv input_file_3390.csv input_file_4500.csv
input_file_1200.csv input_file_2340.csv input_file_3420.csv input_file_4530.csv
input_file_1230.csv input_file_2370.csv input_file_3450.csv input_file_4560.csv
input_file_1260.csv input_file_240.csv  input_file_3480.csv input_file_4590.csv
input_file_1290.csv input_file_2400.csv input_file_3510.csv input_file_4620.csv
input_file_1320.csv input_file_2430.csv i

### <span style="color:green;">3. **将给出的样本数据扩展为三维tensor，并在时间维度上进行一维卷积，卷积核为（1*3），步长为1，并对卷积的数据进行维度还原。**
### **该题考查一维卷积，反卷积，分值为30分**</span>
>提示：卷积还原过程可以不对数据的数值进行还原，只对数据维度进行还原。   

>一维卷积是沿着时间轴进行卷积，需要用permute()进行维度位置变换



In [ ]:
import pandas as pd 
import torch 
import torch.nn as nn 
from torch.autograd import Variable 
sample_data = pd.read_csv("sample_data_A100159.csv")
#################################
#  CODING HERE
#################################






### <span style="color:green;">4. **对给出的样本文件中的所有特征列做normalization，映射至（0-255）的区间，从第一行开始每隔200条取一次整行数据，并将最终取出的数据并以PNG格式的图片输出。**
### **该题考查normalization及数据预处理基础，分值为20分**</span>
>提示：将数组保存为图片

In [ ]:
import pandas as pd 
import numpy as np 
import random 
from PIL import Image 
sample_data = pd.read_csv("sample_data_A100159.csv")

print(sample_data)
for col in sample_data.columns:
    print(col)
    col_min = min(sample_data[col])
    col_max = max(sample_data[col])
    print(col_max)
    print(col_min)
    try:
        sample_data[col] = sample_data[col].apply(lambda x: int((x -  col_min) / (col_max -col_min)  * 255 ) )
    except ZeroDivisionError:
        sample_data[col] = 0
print(sample_data)

arr = []
for i in range(0,len(sample_data),200):
    arr.append(list(sample_data.iloc[i,:].values))
print(arr)
#################################
#  CODING HERE
#################################

In [ ]:
from PIL import imshow

In [ ]:
from PIL import Image 

In [ ]:
# im = Image.image(arr)
pd.DataFrame(arr).values

In [ ]:
from PIL import Image
im = Image.fromarray(arr)
im.save("out.jpeg")

In [ ]:
from PIL import Image
im = Image.fromarray(pd.DataFrame(arr).values)
im.save("out.jpeg")

      BMS_BattCurr  BMS_RMC_CellVoltMax  BMS_SOC  BMS_BattVolt  \
0              0.0                 3.64     36.0         334.5   
1              0.0                 3.64     36.0         334.5   
2              0.0                 3.64     36.0         334.5   
3              0.0                 3.64     36.0         334.5   
4              0.0                 3.64     36.0         334.5   
...            ...                  ...      ...           ...   
4900         -26.0                 4.23     99.0         388.2   
4901         -26.0                 4.23     99.0         388.5   
4902         -25.9                 4.23     99.0         388.2   
4903         -25.8                 4.23    100.0         388.2   
4904         -14.0                 4.23    100.0         388.0   

      BMS_RMC_ModuleTempMax  BMS_RMC_ModuleTempMin  abstime  VehTotDistance  
0                      31.0                   28.0        0           10000  
1                      31.0                   28.0 

In [37]:
from PIL import imshow

ImportError: cannot import name 'imshow' from 'PIL' (/Users/andrew/django-vue-admin/venv/lib/python3.9/site-packages/PIL/__init__.py)

In [38]:
from PIL import Image 

In [47]:
# im = Image.image(arr)
pd.DataFrame(arr).values

array([[255,   0,   0,   0, 191, 127,   0,   0],
       [ 11,  25,   3,  29, 191, 127,  10,   0],
       [ 11,  34,  19,  36, 255, 191,  20,   0],
       [  9,  43,  31,  43, 255, 255,  31,   0],
       [  7,  51,  47,  53, 255, 191,  41,   0],
       [  7,  60,  59,  63, 255, 191,  51,   0],
       [  6,  73,  75,  76, 255, 191,  62,   0],
       [  4,  90,  87,  92, 255, 191,  72,   0],
       [  3, 103, 103, 106, 255, 191,  83,   0],
       [  0, 121, 115, 122, 255, 191,  93,   0],
       [ 15, 133, 127, 136, 191, 127, 103,   0],
       [ 14, 151, 143, 154, 191, 127, 114,   0],
       [  9, 177, 163, 181, 191, 127, 124,   0],
       [ 11, 185, 167, 187, 191, 127, 135,   0],
       [ 81, 190, 179, 191, 127,  63, 145,   0],
       [ 79, 203, 191, 205, 127,  63, 155,   0],
       [ 74, 216, 203, 218,  63,   0, 166,   0],
       [107, 224, 215, 226,  63,   0, 176,   0],
       [123, 233, 227, 233,  63,   0, 187,   0],
       [190, 233, 235, 234,   0,   0, 197,   0],
       [192, 233, 23

In [45]:
from PIL import Image
im = Image.fromarray(arr)
im.save("out.jpeg")

AttributeError: 'list' object has no attribute '__array_interface__'

In [48]:
from PIL import Image
im = Image.fromarray(pd.DataFrame(arr).values)
im.save("out.jpeg")

TypeError: Cannot handle this data type: (1, 1), <i8